In [1]:
from pandas.io.parsers import read_csv
import numpy as np

np.random.seed(0) # REMOVE FOR FINAL SOLUTION


In [2]:
#Data Splitted.
def splitData(data):
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    testCount = int(shuffled.shape[0]*1/3)
    trainCount = shuffled.shape[0]-testCount

    trainSet = shuffled[0:trainCount]
    testSet = shuffled[trainCount:]

    return trainSet, testSet

def separateInputOutput(dataset):
    outputData = dataset[:,3] # Output
    inputData = np.concatenate((dataset[:,0:3],dataset[:,4:7],dataset[:,8:10]),axis=1) # Inputs
    return inputData, outputData

#ML ALG
def initializeWeights(x,y):
    N = x.shape[0]
    
def normaliseData(dataArray, normType = "range"):
    normalisedDataList = []

    if normType == "range":
        maxVal = dataArray.max()
        minVal = dataArray.min()
        for val in dataArray:
            normalisedDataList.append((val-minVal)/(maxVal-minVal))
        return np.array(normalisedDataList)

    elif normType == "z":
        mean = dataArray.mean()
        stdDev =  getStandardDeviation(dataArray)
        for val in dataArray:
            normalisedDataList.append((val-mean)/stdDev)
        return np.array(normalisedDataList)

def getStandardDeviation(dataArray):
    n = dataArray.shape[0]
    mean = dataArray.mean()
    sumOfSquareDiffs = 0
    for val in dataArray:
        sumOfSquareDiffs += (val - mean)**2

    variance = sumOfSquareDiffs/(n-1)
    stdDev = variance**(1/2)
    return stdDev


In [11]:
#Reading in data
dataframe = read_csv("beer.txt",delim_whitespace=True,header=None)

# Preparing Data
dataset = dataframe.values
trainSet, testSet = splitData(dataset)

testIn, testOut = separateInputOutput(testSet)
trainIn, trainOut = separateInputOutput(trainSet)

N =trainIn.shape[0]
Wi = 1/N

#running ml alg.
train(trainIn,trainOut) #ML ALG RAN HERE 


[array([43.35840708, 0.144433099, 1.222727273, 3.744615385, 16.45,
       10.37236842, 17.4, 70.13857143], dtype=object), array([38.35840708, 0.28217077100000004, 4.255454545, 4.070769231, 18.3,
       4.0466842110000005, 6.72, 62.84571429], dtype=object), array([37.65044248, 0.244681179, 2.4136363640000003, 3.870769231, 16.29,
       3.219052632, 6.96, 57.21428571], dtype=object), array([40.34955752, 0.197146734, 0.629090909, 3.904615385, 16.53,
       5.315473684, 11.52, 61.59142857], dtype=object), array([45.52654867, 0.189464469, 1.750909091, 3.8892307689999996, 16.36,
       6.581526316000001, 11.76, 54.91142857], dtype=object), array([40.03982301, 0.347918901, 2.859090909, 4.1784615380000005, 17.58,
       7.340842105, 8.88, 74.88714286], dtype=object), array([38.13716814, 0.233793401, 4.762727273, 4.147692308, 16.67,
       6.252105263, 6.84, 73.39428571], dtype=object), array([40.52654867, 0.237383712, 1.767272727, 4.033846154, 16.01,
       13.27905263, 13.44, 48.32857143], dt

In [4]:
def getUniqueClassCount(listToCount):
    classes = []
    for item in listToCount:
        if item not in classes:
            classes.append(item)

    counts = []
    for itemClass in classes:
        counts.append(listToCount.count(itemClass))
    return counts

def getGiniLeaf(counts):
    total = np.sum(counts)
    giniLeaf = 1
    for count in counts:
        giniLeaf -= (count/total)**2
    return giniLeaf

def getGiniNode(sortedList, threshold):
        lessThan = []
        greaterThan = []

        for i in range(len(sortedList)):
            x = sortedList[i][0]
            # print(x, threshold)
            if x < threshold:
                lessThan.append(sortedList[i][1])
            else:
                greaterThan.append(sortedList[i][1])

        countsLT = getUniqueClassCount(lessThan)
        countsGT = getUniqueClassCount(greaterThan)
        giniLT = getGiniLeaf(countsLT)
        giniGT = getGiniLeaf(countsGT)
        # print(giniLT,giniGT)
        # print(np.sum(countsLT),np.sum(countsGT))
        totalCount = len(sortedList)
        # print(totalCount)
        giniNode = ((np.sum(countsLT)/totalCount)*giniLT) + ((np.sum(countsGT)/totalCount)*giniGT)
        # print(giniNode)
        return giniNode


In [5]:
def getSplitValue(trainIn, trainOut, attributeIndex):
    attribute = trainIn[:, attributeIndex].copy() # Output
    combined = []
    for j,sample in enumerate(attribute):
        combined.append((sample,trainOut[j]))
    attribute.sort()
    sortedList = [tuple for x in attribute for tuple in combined if tuple[0] == x]


    thresholds = []
    for j in range(len(attribute)-1):
        thresholds.append((attribute[j]+attribute[j+1])/2)

    giniNodes = []
    for threshold in thresholds:
        giniNodes.append(getGiniNode(sortedList, threshold))
    # print(giniNodes)
    minNodeGini = min(giniNodes)
    minGiniIndex = giniNodes.index(minNodeGini)
    splitVal = thresholds[minGiniIndex]
    return splitVal, minNodeGini
   

In [8]:
def train(trainIn,trainOut):
    ginis = []
    splitVals = []
    for i in range(trainIn.shape[1]):
        splitVal, minGini = getSplitValue(trainIn,trainOut, i)
        ginis.append(minGini)
        splitVals.append(splitVal)
        # print(splitVal, minGini)
    # print("Root is input data attribute", ginis.index(min(ginis)))
    branchData(testIn,testOut,ginis.index(min(ginis)),splitVals[ginis.index(min(ginis))]) #

In [10]:
def branchData(X, y, splitAttributeIndex, splitValue):
    lessThan_X = []
    lessThan_y = []
    greaterThan_X = []
    greaterThan_y = []
    
    for i in range(len(X)):
        if X[i,splitAttributeIndex] < splitValue:
            lessThan_X.append(X[i,:])
            lessThan_y.append(y[i])
        else:
            greaterThan_X.append(X[i,:])
            greaterThan_y.append(y[i])
    print(lessThan_X)